In [26]:
using JuMP
using Gurobi
using Plots
gr()
using AxisArrayVariables

INFO: Precompiling module AxisArrayVariables.


In [162]:
include("piecewise.jl")
include("mpc.jl")
include("learning.jl")

function JuMP.solve(mm::MPC.MPCModel, x0::Vector)
    JuMP.fix.(mm.x[:, 1], x0)
    status = solve(mm.model; suppress_warnings=true)
    utraj = pf.PiecewiseFunction(mm.times, [t -> getvalue.(mm.u[:, j]) for j in 1:size(mm.u, 2)])
    status, utraj
end

function MPC.output(controller::PiecewiseFunctions.PiecewiseFunction, t, state) 
    controller(t)
end

LoadError: LoadError: UndefVarError: AbstractDataProvider not defined
while loading /Users/rdeits/locomotion/explorations/learning-mpc/learning.jl, in expression starting on line 12

In [155]:
mass = 1.
l = 1.
g = 10.
N = 4
A = [0. 1.;
     g/l 0.]
B = [0 1/(mass*l^2.)]'
Δt = .1
sys = MPC.discretize(MPC.CTLinearSytstem(A, B), Δt)

x_max = [pi/6, pi/20/(N*Δt)]
x_min = -x_max
u_max = [mass*g*l*pi/8.]
u_min = -u_max
times = 0:Δt:N*Δt

0.0:0.1:0.4

In [157]:
x0 = x_min .+ rand(2) .* (x_max .- x_min)
mm = MPC.MPCModel(sys, N)
setsolver(mm.model, GurobiSolver(OutputFlag=0))
setlowerbound.(mm.u, u_min)
setupperbound.(mm.u, u_max)
setlowerbound.(mm.x[:, 2:end], x_min)
setupperbound.(mm.x[:, 2:end], x_max)
status, utraj = solve(mm, x0)
states = MPC.simulate(sys, utraj, x0, times)
for i in 1:length(states)
    @assert states[i] ≈ getvalue.(mm.x[:, i])
end

In [160]:
mm = MPC.MPCModel(sys, N)
setsolver(mm.model, GurobiSolver(OutputFlag=0))
setlowerbound.(mm.u, u_min)
setupperbound.(mm.u, u_max)
setlowerbound.(mm.x[:, 2:end], x_min)
setupperbound.(mm.x[:, 2:end], x_max)

samples = Learning.Sample{2, 1}[]
num_samples = 1000
for i in 1:num_samples
    while true
        x0 = x_min .+ rand(2) .* (x_max .- x_min)
        status, utraj = solve(mm, x0)
        if status == :Optimal
            push!(samples, Learning.Sample{2, 1}(x0, getvalue.(mm.u[:, 1])))
            break
        end
    end
end

        

In [ ]:
using MXNet

